In [15]:
import pandas as pd
import praw
import pickle
import concurrent
import threading
import time

In [6]:
client_id = 'FD6fGd4uEID1MQ'
client_secret = 'PVC2kUrAcVVrn0GvT5EUAtesXR5_WQ'
user_agent = 'scrapeWSB'

reddit = praw.Reddit(client_id=client_id, client_secret = client_secret, user_agent = user_agent)

In [7]:
wsb = reddit.subreddit('WallStreetBets')

In [8]:
url = 'https://www.reddit.com/r/wallstreetbets/comments/iblrnn/what_are_your_moves_tomorrow_august_18_2020/'

sub = reddit.submission(url=url)

In [9]:
comments = list()
from praw.models import MoreComments
for post in sub.comments:
    if isinstance(post, MoreComments):
        continue
        print(post)
    comments.append([post.body, post.score, post.id, post.subreddit, post.created])

In [10]:
posts = pd.DataFrame(comments,columns=['text', 'score', 'id', 'subreddit', 'created'])
posts

,text,score,id,subreddit,created
0,#S&P to join TSLA,204,g1wilk0,wallstreetbets,1.597724e+09
1,I still can't believe that just over a year ag...,93,g1wybfd,wallstreetbets,1.597729e+09
2,For a company with the market cap of Tesla to ...,82,g1wl07n,wallstreetbets,1.597725e+09
3,Fuck you autists. I decided to gamble my money...,83,g1wr78n,wallstreetbets,1.597727e+09
4,We should rent out an abandoned autobody shop ...,83,g1xe83q,wallstreetbets,1.597737e+09
...,...,...,...,...,...
414,TRUST THE ORACLE 🔮🧙🏼‍♂️,9,g1ye3t3,wallstreetbets,1.597758e+09
415,"Gold and silver boutta moon rn, has a pretty s...",10,g1yeh7n,wallstreetbets,1.597759e+09
416,[deleted],9,g1yfaf9,wallstreetbets,1.597759e+09
417,[deleted],10,g1yhvuo,wallstreetbets,1.597762e+09


In [11]:
with open('tickers.pkl', 'rb') as f:
    tickers = pickle.load(f)

tickers = [{'ticker': f' {ticker} ', 'occur': 0} for ticker in tickers]

tickers = pd.DataFrame(tickers)

In [12]:
class database():
    def __init__(self, tick):
        self.tickers = tick
        self._lock = threading.Lock()
    def update(self, post):
        for index, col in self.tickers.iterrows():
            if col['ticker'] in post:
                local_copy = self.tickers.loc[index, 'occur']
                local_copy += 1
                time.sleep(0.1)
                self.tickers.loc[index, 'occur'] = local_copy

In [13]:
db = database(tickers)

In [16]:
start = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    for post in posts['text'].to_list():
        executor.submit(db.update, post)
end = time.time()
elapsed = round((end-start) / 60)
print(f'{elapsed} minutes')

8 minutes


In [17]:
sorted_tickers = db.tickers.sort_values(by = 'occur', axis = 0, ascending=False)

In [18]:
banned = ['ABLE', 'FLEX', 'BOTH', 'LOW', 'TELL', 'FIND', 'WISH', 'X', 'TRUE', 'MOST', 'GAIN', 'RSI', 'HOPE', 'A', 'ON', 'FOR', 'IT', 'BE', 'AT', 'ALL', 'WHEN', 'INTO', 'SO', 'OR', 'NOW', 'MAKE', "AN", 'HAS', 'SEE', 'BEEN', 'BY', 'GO', 'SOME', 'HE', 'CAP', 'LOVE', 'AM', 'ANY', 'ELSE', 'TECH', 'FREE', 'POST', 'NEXT', 'MAN', 'WAKE', 'HOME', 'U', 'PM', 'AGO', 'LIFE', 'EVER', 'ATH', 'BIG', 'PUMP', 'KIDS', 'CAR', 'OPEN']

In [19]:
def check_banned(x):
    if x not in banned:
        return x

In [20]:
banned = [f' {x} ' for x in banned]

In [21]:
mask = sorted_tickers['ticker'].apply(check_banned)

In [37]:
mask = mask.dropna()[:10]

In [72]:
dicts = [sorted_tickers[sorted_tickers['ticker'] == ticker].to_dict() for ticker in mask]

In [81]:
ret = [{'ticker': list(a['ticker'].values())[0], 'occur': list(a['occur'].values())[0]} for a in dicts]

In [80]:
import json

In [82]:
json.dumps(ret)

'[{"ticker": " TSLA ", "occur": 50}, {"ticker": " AAPL ", "occur": 7}, {"ticker": " WMT ", "occur": 3}, {"ticker": " RH ", "occur": 2}, {"ticker": " NVDA ", "occur": 1}, {"ticker": " MS ", "occur": 1}, {"ticker": " DOCU ", "occur": 1}, {"ticker": " PRPL ", "occur": 1}, {"ticker": " OSTK ", "occur": 1}, {"ticker": " HD ", "occur": 1}]'